# Wrapping a simple library

We here aim at presenting the interactive wrapping workflow.
For the sake of simplicity, we consider an artificial example of *C++* library.


## The *C++* library

This simple library contains one header (`simple_library/binomial.h` see below) and one source code file (`simple_library/binomial.cpp`).
This artificial *C++* library implements mass computation for binomial distributions (`BinomialDistribution::pmf`).
If an user try to set the probability of the binomial distribution `BinomialDistribution::_pi`) to values outside the interval $[0,1]$, a `ProbabilityError` exception is thrown.

In [1]:
from pygments import highlight
from pygments.lexers import CppLexer
from pygments.formatters import HtmlFormatter
from IPython.core.display import HTML

with open('simple_library/binomial.h', 'r') as filehandler:
    header = HTML(highlight(filehandler.read(), CppLexer(),
                         HtmlFormatter(full=True)))
header

In order to generate *Python* bindings for this *C++* library we need:

* To compile the source code.

In [2]:
import subprocess
output = subprocess.check_output(['g++', '-o', 'simple_library/binomial.os',
                 '-c', '-x', 'c++', '-std=c++0x', '-Wwrite-strings',
                 '-fPIC', '-Isimple_library', 'simple_library/binomial.cpp'])

* To make a shared library.

In [3]:
output = subprocess.check_output(['g++', '-o', 'simple_library/libbinomial.so',
                                  '-shared', 'simple_library/binomial.os'])

* To import the shared library.

In [8]:
from ctypes import cdll
lib1 = cdll.LoadLibrary('simple_library/libbinomial.so')

## Generation of *Python* bindings

Once the *C++* library has been compiled, we can proceed to the generation of its *Python* bindings using **AutoWIG**.

In [ ]:
from autowig import autowig

The header is first parsed with relevant compilation flags.

In [1]:
asg = autowig.AbstractSemanticGraph()
asg = autowig.parser(asg,
                     ['./simple_library/binomial.h'],
                     ['-x', 'c++', '-std=c++0x', '-Isimple_library'])

NameError: name 'autowig' is not defined

Since most of **AutoWIG** guidelines are respected, the `default` `controller` implementation is thus suitable.

In [ ]:
asg = autowig.controller(asg)

In order to wrap the library we need to select the `boost_python_internal` `generator` implementation.

In [ ]:
autowig.generator.plugin = 'boost_python_internal'

The **Boost.Python** module name chosen is `'./test/binomial/module.cpp'` and no decorator has to be written (`None`).

In [ ]:
wrappers = autowig.generator(asg,
                             module = './simple_library/_libbinomial.cpp',
                             decorator = None)

The wrappers are only generated in-memory, we therefore need to write them on the disk to complete the process.

In [ ]:
for wrapper in wrappers:
    wrapper.write()

Once the wrappers are written to disk, we need to compile the *Python* bindings using **SCons**.

In [ ]:
autowig.scons('test')

## Usage of *Python* bindings

And we can hereafter use the *C++* library in the *Python* interpreter

In [3]:
import sys
import os
import autowig

libpath = os.path.abspath('simple_library')
if sys.platform == 'win32':
    os.environ['PATH'] += ':' + libpath
elif sys.platform == 'darwin':
    os.environ['DYLD_LIBRARY_PATH'] += ':' + libpath
else:
    os.environ['LD_LIBRARY_PATH'] += ':' + libpath
execmd = sys.executable
argv = sys.argv + [os.environ]
os.execle(execmd, execmd, *argv)

ImportError: No module named autowig

In [ ]:
from simple_library import *
binomial = BinomialDistribution(1,.5)

In [ ]:
binomial.pmf(0)

In [ ]:
binomial.pmf(1)

In [ ]:
binomial.n = 0
binomial.pmf(0)

In [ ]:
binomial.set_pi(1.1)
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
ProbabilityError: a probability must be in the interval [0,1]